In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery
import glob
import gzip
from tqdm import tqdm

dir_ = '/data/pj20/compound/general/'

In [3]:
import pickle
with open('./cid_set.pkl', 'rb') as f:
    cid_set = pickle.load(f)

In [4]:
from tqdm import tqdm
from collections import defaultdict

def retrieve(triple_strs, relation):
    triple_set = set()
    t_buffer = defaultdict(list)
    h_buffer = defaultdict(list)
    for triple_str in tqdm(triple_strs):
        if ' ,\n\t\t' in triple_str:
            items = triple_str.split(' ,\n\t\t')
            hr_t0 = items[0]
            h, r, t0 = hr_t0.split('\t')
            h = h.replace('compound:CID', '')
            t0 = t0.replace('compound:CID', '')

            if h in cid_set and t0 in cid_set:
                triple_set.add((h, relation,t0))

            elif h in cid_set and t0 not in cid_set:
                t_buffer[t0].append(h)
            
            elif h not in cid_set and t0 in cid_set:
                h_buffer[h].append(t0)


            tail_num = len(items) - 1
            for tail in range(tail_num):
                t = items[tail+1].replace('compound:CID', '')
                if h in cid_set and t in cid_set:
                    triple_set.add((h, relation, t))

                elif h in cid_set and t not in cid_set:
                    t_buffer[t].append(h)
                
                elif h not in cid_set and t in cid_set:
                    h_buffer[h].append(t)
        else:
            try:
                items = triple_str.split('\t')
                h = items[0].replace('compound:CID', '')
                t = items[2].replace('compound:CID', '')
                if h in cid_set and t in cid_set:
                    triple_set.add((h, relation, t))

                elif h in cid_set and t not in cid_set:
                    t_buffer[t].append(h)
                
                elif h not in cid_set and t in cid_set:
                    h_buffer[h].append(t)
            except:
                continue
                
    return triple_set, h_buffer, t_buffer

***=============================== compound2component ===============================***

In [15]:
with open(dir_ + 'pc_compound2component.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [16]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_component")

100%|██████████| 23769818/23769818 [00:28<00:00, 839826.20it/s] 


In [17]:
cnt = 0

for key in h_buffer.keys():
    if len(h_buffer[key]) > 30:
        cnt+=1
        for t in h_buffer[key]:
            triple_set.add((key, 'has_component', t))
print(cnt, len(triple_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_component.txt', 'w') as f:
    f.write(out_str)

444 24372


***=============================== compound2parent ===============================***

In [18]:
with open(dir_ + 'pc_compound2parent.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [19]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_parent")

100%|██████████| 6943268/6943268 [00:04<00:00, 1525561.63it/s]


In [31]:
cnt = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 2:
        cnt+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'has_parent', key))
print(cnt, len(triple_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_parent.txt', 'w') as f:
    f.write(out_str)

35 4452


***=============================== compound2stereoisomer ===============================***

In [12]:
with open(dir_ + 'pc_compound2stereoisomer_000001.ttl', 'r') as f:
    lines = f.read()

triple_strs_1 = lines.split(' .\n')[2:]

with open(dir_ + 'pc_compound2stereoisomer_000002.ttl', 'r') as f:
    lines = f.read()

triple_strs_2 = lines.split(' .\n')[2:]

with open(dir_ + 'pc_compound2stereoisomer_000003.ttl', 'r') as f:
    lines = f.read()

triple_strs_3 = lines.split(' .\n')[2:]

triple_strs = triple_strs_1 + triple_strs_2 + triple_strs_3

In [13]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_stereoisomer")

100%|██████████| 24904630/24904630 [00:34<00:00, 724908.81it/s] 


In [14]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 10:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'has_stereoisomer', key))
        cid_set.add(key)

cnt_h = 0
for key in h_buffer.keys():
    if len(h_buffer[key]) > 10:
        cnt_h+=1
        for t in h_buffer[key]:
            triple_set.add((key, 'has_stereoisomer', t))
        cid_set.add(key)

print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_stereoisomer.txt', 'w') as f:
    f.write(out_str)

1232 1232 57156 44370


***=============================== compound2same_connectivity ===============================***

In [48]:
with open(dir_ + 'pc_compound2sameconnectivity.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [56]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_same_connectivity")

100%|██████████| 2821372/2821372 [00:01<00:00, 1490031.91it/s]


In [59]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 10:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'has_same_connectivity', key))
        cid_set.add(key)

cnt_h = 0
for key in h_buffer.keys():
    if len(h_buffer[key]) > 10:
        cnt_h+=1
        for t in h_buffer[key]:
            triple_set.add((key, 'has_same_connectivity', t))
        cid_set.add(key)

print(cnt_t, cnt_h, len(triple_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_connectivity.txt', 'w') as f:
    f.write(out_str)

255 255 7484


***=============================== compound2isotopologue ===============================***

In [15]:
with open(dir_ + 'pc_compound2isotopologue.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [16]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_isotopologue")

100%|██████████| 368953/368953 [00:02<00:00, 174004.29it/s]


In [17]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 3:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'has_isotopologue', key))
        cid_set.add(key)

cnt_h = 0
for key in h_buffer.keys():
    if len(h_buffer[key]) > 3:
        cnt_h+=1
        for t in h_buffer[key]:
            triple_set.add((key, 'has_isotopologue', t))
        cid_set.add(key)

print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_isotopologue.txt', 'w') as f:
    f.write(out_str)

51 51 700 44421


***=============================== compound2drugproduct ===============================***

In [96]:
with open(dir_ + 'pc_compound2drugproduct.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [97]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="to_drug")

100%|██████████| 17179/17179 [00:00<00:00, 621969.72it/s]


In [101]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 2:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'to_drug', key))


print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    t = t.replace("<http://purl.bioontology.org/ontology/SNOMEDCT/", "SNOMED_").replace(">", "").replace("ns2:", "ns2_")
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_todrug.txt', 'w') as f:
    f.write(out_str)

778 51 3632 44421


***=============================== compound2closematch ===============================***

In [102]:
with open(dir_ + 'pc_compound_closematch.ttl000001.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [104]:
_, _, t_buffer = retrieve(triple_strs, relation="closematch")

100%|██████████| 951092/951092 [00:00<00:00, 1424389.33it/s]


In [116]:
cnt_t = 0

for key in t_buffer.keys():
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'closematch', key))


print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    t = t.replace(":", "_")
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_closematch.txt', 'w') as f:
    f.write(out_str)

36173 51 36173 44421


***=============================== compound2type ===============================***

In [124]:
with open(dir_ + 'pc_compound_type.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [125]:
_, _, t_buffer = retrieve(triple_strs, relation="type")

100%|██████████| 151979/151979 [00:00<00:00, 973600.04it/s]


In [137]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 2:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'type', key))


print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    t = t.replace("<http://purl.bioontology.org/ontology/SNOMEDCT/", "SNOMED_").replace(">", "").replace(":", "_")
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_totype.txt', 'w') as f:
    f.write(out_str)

1164 51 5323 44421


***=============================== compound2descriptor ===============================***

In [142]:
from tqdm import tqdm

triple_strs = []

for i in tqdm(range(1, 161)):
    if len(str(i)) == 1:
        idx = "00" + str(i)
    if len(str(i)) == 2:
        idx = "0" + str(i)
    if len(str(i)) == 3:
        idx = str(i)
    
    file_name = dir_ + f'pc_compound2descriptor_000{idx}.ttl'

    with open(file_name, 'r') as f:
        lines = f.read()

    triple_strs += lines.split(' .\n')[2:]

100%|██████████| 160/160 [03:30<00:00,  1.32s/it]


In [147]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_descriptor")

100%|██████████| 114013741/114013741 [12:44<00:00, 149159.81it/s]


In [153]:
cnt_t = 0

for key in t_buffer.keys():
    for h in t_buffer[key]:
        triple_set.add((h, 'has_descriptor', key))


print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_descriptor.txt', 'w') as f:
    f.write(out_str)

0 51 1038933 44421


***=============================== compound2pathway ===============================***

In [27]:
with open(dir_ + 'pc_pathway.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [32]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="in_pathway")

100%|██████████| 494606/494606 [00:01<00:00, 325118.41it/s]


In [37]:
cnt_t = 0

triple_set = set()
for key in h_buffer.keys():
    tmp_set = set()
    for cid in h_buffer[key]:
        if cid in cid_set:
            tmp_set.add((cid, 'in_pathway', key.replace('pathway:', '')))
        if len(tmp_set) > 8:
            triple_set.update(tmp_set)

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_pathway.txt', 'w') as f:
    f.write(out_str)